In [1]:
#import des librairies

##pour les graphes
import networkx as nx 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import IsolationForest as IF
from networkx.algorithms.bipartite import weighted_projected_graph
#greedy is for clauset-newman
from networkx.algorithms.community import louvain_communities, greedy_modularity_communities, naive_greedy_modularity_communities, girvan_newman
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from random import sample
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.spatial.distance import jaccard
from collections import Counter
from sklearn.cluster import KMeans

In [2]:
df= pd.read_csv("customer_to_part.csv")

In [3]:
df= df[["P_PARTKEY", "C_CUSTKEY", "QUANTITY"]]
df.shape

(23996670, 3)

In [4]:
#
def preprocessing(df, n_client= 10000, n_produit= 10000):
    df0= df.drop_duplicates()
    #gestion des valeurs inconnues
    if df0.isna().sum().sum()!=0:
        df0=df0.dropna()
    #Faisons une selection des columns
    #choix de n_client clients
    clt= sample(list(df0["C_CUSTKEY"].unique()), n_client)
    #selection des n_produit produits les plus achetés
    part=df0.groupby("P_PARTKEY")["QUANTITY"].aggregate('sum').sort_values(ascending=False)[:n_produit]
    prod= list(part.index)
    #
    df1= df0[df0["C_CUSTKEY"].isin(clt)]
    df1= df1[df1["P_PARTKEY"].isin(prod)]
    #pivotage du tableau
    pivot=df1.groupby(["C_CUSTKEY", "P_PARTKEY"])["QUANTITY"].aggregate('sum').unstack()
    pivot= pivot.replace(np.nan, 0)
    #centrons et reduisons les quantités
    sc= StandardScaler()
    matricePC= sc.fit_transform(pivot.values)
    # obtenir les libellés
    clients= np.array(pivot.index)
    produits=np.array(pivot.columns)
    return matricePC, clients, produits, pivot, df1
    

In [5]:
mat, clt, prod, pivot, selecteddf= preprocessing(df)

In [6]:
#groupe de clients sur qui concerne l'étude
clt

array(['C100007', 'C100039', 'C100048', ..., 'C99968', 'C99980', 'C99989'],
      dtype=object)

In [7]:
#fonction qui retourne la liste des 10 clients les plus proches du client clt 
def nearest(clt, matrice, clients, produits, similarity="cosine"):
    proche=[]
    try:
        #l'index du client dans le dataframe
        idx= np.where(clients== clt)[0][0]
        #avec la similarité de cosinus
        if similarity == "cosine":
            #similarité de cosinus sur les clients
            cos= cosine_similarity(matrice)
            #recuperer et classer les index clients du moins proche au plus proche à clt
            groupe= np.argsort(cos[idx])[-10:-1]
            #récupérer les noms des clients grace à leur index
            proche= clients[groupe]
        #avec la distance euclidéenne
        elif similarity == "euclidean":
            groupe={}
            lk= linkage(matrice, metric="euclidean", method="ward")
            #clustering des clients en fonction des la distance
            cluster= fcluster(lk, t=200, criterion="distance")
            #grouper les clients par cluster
            for i, x in enumerate(cluster):
                if x in groupe.keys():
                    groupe[x].append(clients[i])
                else:
                    groupe[x]= [clients[i]]
            #récupèrer les cluster auquel appartient le groupe 
            for val in groupe.values():
                if clt in val:
                    proche.extend(list(val))
                    break
        elif similarity == "pearson":
            #correlation de pearson
            corr= np.corrcoef(mat)
            #recuperer l'indice du client afin d'obtenir sa ligne correspondante dans cos
            groupe= np.argsort(corr[idx])[-10:-1]
            proche= clients[groupe]
        return proche
    except Exception:
        return "check the client you give as parameter"
    

In [8]:
#comparaison de similarité 1 par rapport à 2 faite sur n clients
def comparaison(similarite1, similarite2, n=3):
    difference=[]
    for i in range(n):
        liste1= nearest(clt[i], mat, clt, prod, similarite1)
        liste2= nearest(clt[i], mat, clt, prod, similarite2)
        difference.append(sum([1 if x in liste2 else 0 for x in liste1 ])/len(liste2))
    moy= (sum(difference)/len(difference))*100
    return "ces deux similarités sont à "+ str(moy)+ " % compatibles"

In [9]:
comparaison("euclidean", "pearson")

'ces deux similarités sont à 62.96296296296296 % compatibles'

In [10]:
nearest(clt[0], mat, clt, prod, "cosine")

array(['C3359', 'C59092', 'C92302', 'C95887', 'C56369', 'C117058',
       'C49865', 'C8090', 'C4999'], dtype=object)

In [11]:
nearest(clt[0], mat, clt, prod, "pearson")

array(['C3359', 'C59092', 'C92302', 'C95887', 'C56369', 'C117058',
       'C49865', 'C8090', 'C4999'], dtype=object)

In [12]:
def proposition(client, method="cosine"):
    #liste des clients proches de client passé en paramètres
    near=list(nearest(client, mat, clt, prod, method))
    idx=[]
    #pour chaque client dans la liste de clients proches
    for c in near:
        #on récupère son index
        idx.append(np.where(clt== c)[0][0])
    #on fait un traitement sur les produits acheter en commun par ces clients
    #d'abord on recupère les produits les plus achétés en terme de quantité grace aux index
    res= np.sum(pivot.values[idx], axis=0)
    #--puis on les classe par ordres croissant et on récupère les 10 derniers les plus achetés
    arg= np.argsort(res)[-10:-1]
    final=prod[arg]
    print(pivot[final].iloc[np.where(clt== client)[0][0]])
    return final
    

In [13]:
proposition(clt[0])

P_PARTKEY
P222614     0.0
P376724     0.0
P28617      0.0
P87355      0.0
P52921      0.0
P260105     0.0
P31050      0.0
P233428    36.0
P74461     31.0
Name: C100007, dtype: float64


array(['P222614', 'P376724', 'P28617', 'P87355', 'P52921', 'P260105',
       'P31050', 'P233428', 'P74461'], dtype=object)

In [14]:
#clustering avec les k moyens
km= KMeans(n_clusters=50, random_state=123).fit(mat)

In [15]:
#prediction des clusters
pred= km.predict(mat)
group=list(pred)
#clustering de clients
clust={}
#rassemblement des clients par cluster
for i, val in enumerate(group):
    if val in clust.keys():
        clust[val].append(clt[i])
    else:
        clust[val]=[clt[i]]
print(list(clust.values()))

[['C100007', 'C100039', 'C100048', 'C100063', 'C10007', 'C100070', 'C100082', 'C100097', 'C100126', 'C100144', 'C100145', 'C100151', 'C100159', 'C100163', 'C100186', 'C100198', 'C100208', 'C100210', 'C100231', 'C100237', 'C100241', 'C100243', 'C100250', 'C100253', 'C100268', 'C100270', 'C100286', 'C10030', 'C100324', 'C100343', 'C100361', 'C100364', 'C100375', 'C100379', 'C100402', 'C100411', 'C100420', 'C100427', 'C100456', 'C10046', 'C100501', 'C100508', 'C100517', 'C100538', 'C100544', 'C100555', 'C100576', 'C100580', 'C100588', 'C1006', 'C100600', 'C100618', 'C100628', 'C100643', 'C100660', 'C100679', 'C100681', 'C100687', 'C100688', 'C100697', 'C100708', 'C100715', 'C100733', 'C100738', 'C100775', 'C100813', 'C100816', 'C100817', 'C100822', 'C100829', 'C100834', 'C100840', 'C100843', 'C100859', 'C100864', 'C100868', 'C100897', 'C100913', 'C100916', 'C100937', 'C100939', 'C100954', 'C100958', 'C101012', 'C101014', 'C101023', 'C101033', 'C101038', 'C101048', 'C101056', 'C101078', 'C

In [16]:
#clustering par la méthode des graphes
g= nx.Graph()
g.add_weighted_edges_from(selecteddf.values)

In [17]:
#partitionnement de Louvain
lv= louvain_communities(g)

In [18]:
louvain_clients=[]
louvain_produits=[]
#scinder les clusters en clients et produits d'autres parts
for sg in lv:
    louvain_clients.append([i for i in sg if i.startswith("C")])
    louvain_produits.append([i for i in sg if i.startswith("P")])
print("Avec louvain on a", len(lv), "partitions")

Avec louvain on a 52 partitions


In [19]:
#partitionnement de clauset-newman
cn= greedy_modularity_communities(g)

In [20]:
newman_clients=[]
newman_produits=[]
#scinder les clusters en clients et produits d'autres parts
for sg in cn:
    newman_clients.append([i for i in sg if i.startswith("C")])
    newman_produits.append([i for i in sg if i.startswith("P")])
print("Avec clauset-newman on a", len(cn), "partitions")

Avec clauset-newman on a 17 partitions


In [21]:
def compare(list1, list2):
    return sum([1 if x in list2 else 0 for x in list1 ])/len(list2)

def cluster_comparaison(clust1, clust2):
    score=0
    for l1 in clust1:
        for l2 in clust2:
            score+= compare(l1, l2)
    return "c'est deux clusters sont proches de "+ str(score)+ "%"

In [22]:
#comparaison entre newman et louvain
cluster_comparaison(newman_clients, louvain_clients)

"c'est deux cluster sont proches de 52.00000000000013%"